In [67]:
# 1) Install Required Packages
!pip install -q opencv-python
!pip install -q tqdm
!pip install -q scikit-image
!pip install -q einops


# 2) Core Python Imports
import os
import shutil
from pathlib import Path

# 3) Image/Video Processing
import cv2               # OpenCV
import numpy as np


# 4) Metrics (PSNR / SSIM)
from skimage.metrics import peak_signal_noise_ratio, structural_similarity


# 5) Progress Bar
from tqdm import tqdm
# 6) pandas
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
base_dir = Path("/content")

# params
sigma_noise = 25
ksize = 5

# videos
clean_video_path  = base_dir / "data" / "clean.mp4"
noisy_video_path  = base_dir / "data" / "noisy.mp4"
rest_video_path   = base_dir / "data" / "restormer.mp4"
panel5_video_path = base_dir / "data" / "comparison_5panel.mp4"

# frame dirs (data)
clean_frames_dir    = base_dir / "data" / "clean_frames"
noisy_frames_dir    = base_dir / "data" / "noisy_frames"
gaussian_frames_dir = base_dir / "data" / "gaussian_frames"
median_frames_dir   = base_dir / "data" / "median_frames"
qual5_dir           = base_dir / "data" / "qualitative_5panel"

# Restormer dirs
demo_noisy_dir = base_dir / "Restormer" / "demo" / "noisy_frames"
rest_root_dir  = base_dir / "Restormer" / "demo" / "restormer_frames"

# create only dirs we control
for d in [
    clean_frames_dir, noisy_frames_dir,
    gaussian_frames_dir, median_frames_dir,
    qual5_dir,
    demo_noisy_dir
]:
    d.mkdir(parents=True, exist_ok=True)



In [69]:
from google.colab import files

uploaded = files.upload()  # เลือกไฟล์ clean.mp4 จากเครื่อง

# ย้ายไฟล์แรกที่อัปโหลดให้ชื่อ clean.mp4 ใน data/
for fname in uploaded.keys():
    src = Path(fname)
    clean_video_path.parent.mkdir(parents=True, exist_ok=True)
    shutil.move(str(src), str(clean_video_path))
    print("Saved uploaded file as:", clean_video_path)
    break

Saving 1.mp4 to 1.mp4
Saved uploaded file as: /content/data/clean.mp4


In [70]:
!rm -rf /content/Restormer
!git clone https://github.com/swz30/Restormer.git /content/Restormer
!mkdir -p /content/Restormer/Denoising/pretrained_models
!wget https://github.com/swz30/Restormer/releases/download/v1.0/gaussian_color_denoising_blind.pth \
     -O /content/Restormer/Denoising/pretrained_models/gaussian_color_denoising_blind.pth

Cloning into '/content/Restormer'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 312 (delta 74), reused 72 (delta 72), pack-reused 197 (from 2)
Receiving objects: 100% (312/312), 1.55 MiB | 5.19 MiB/s, done.
Resolving deltas: 100% (131/131), done.
--2025-12-26 16:13:33--  https://github.com/swz30/Restormer/releases/download/v1.0/gaussian_color_denoising_blind.pth
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/418793252/ffbfd4cd-b436-4fe1-8e8c-0be81f63ad0b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-26T16%3A54%3A58Z&rscd=attachment%3B+filename%3Dgaussian_color_denoising_blind.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-4

In [71]:
def clear_dir(d):
    if d.exists():
        shutil.rmtree(d)
    d.mkdir(parents=True, exist_ok=True)

# clear only directories we fully control
for d in [
    clean_frames_dir,
    noisy_frames_dir,
    gaussian_frames_dir,
    median_frames_dir,
    demo_noisy_dir,      # input to Restormer
]:
    clear_dir(d)



In [72]:
cap = cv2.VideoCapture(str(clean_video_path))
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))               # รายละเอียดของ VDO ต้นฉบับ
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    clean = frame.astype(np.uint8)
    cv2.imwrite(str(clean_frames_dir / f"{frame_idx:05d}.png"), clean)

    noise = np.random.normal(0, sigma_noise, clean.shape).astype(np.float32)   # noise
    noisy = clean.astype(np.float32) + noise   # add noise to clean file
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    cv2.imwrite(str(noisy_frames_dir / f"{frame_idx:05d}.png"), noisy)

    frame_idx += 1

cap.release()
print("Total frames:", frame_idx)
print("FPS:", fps, "| Size:", width, "x", height)

Total frames: 49
FPS: 8.0 | Size: 720 x 480


In [73]:
# Visualization Utilities (Labels for 5-panel)

labels = ["Original", "Noisy", "Gaussian", "Median", "Restormer"]

def add_labels(frame):
    for i, text in enumerate(labels):
        cv2.putText(
            frame,
            text,
            (i * width + 20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.0,
            (255, 255, 255),
            2
        )
    return frame


In [74]:
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_noisy = cv2.VideoWriter(str(noisy_video_path), fourcc, fps, (width, height))

for i in range(frame_idx):
    f = cv2.imread(str(noisy_frames_dir / f"{i:05d}.png"))
    out_noisy.write(f)

out_noisy.release()
print("Saved noisy video to:", noisy_video_path) # save noise VDO ไปยัง path ที่ตั้งไว้

Saved noisy video to: /content/data/noisy.mp4


In [75]:
for d in [median_frames_dir, gaussian_frames_dir]:
    for f in d.glob("*.png"):
        f.unlink()

noisy_list = sorted(noisy_frames_dir.glob("*.png"))

for img_path in tqdm(noisy_list):
    noisy = cv2.imread(str(img_path))

    den_med   = cv2.medianBlur(noisy, ksize)            # denoise เเบบวิธี clasical method median blur
    den_gauss = cv2.GaussianBlur(noisy, (ksize, ksize), sigmaX=1.0) # denoise เเบบวิธี clasical method gaussian blur

    cv2.imwrite(str(median_frames_dir / img_path.name), den_med)
    cv2.imwrite(str(gaussian_frames_dir / img_path.name), den_gauss)

print("Saved classical denoised frames to:")
print(" -", median_frames_dir)
print(" -", gaussian_frames_dir)

100%|██████████| 49/49 [00:03<00:00, 13.84it/s]

Saved classical denoised frames to:
 - /content/data/median_frames
 - /content/data/gaussian_frames


In [76]:
# ensure directory exists
demo_noisy_dir.mkdir(parents=True, exist_ok=True)

# clear old noisy frames
for f in demo_noisy_dir.glob("*.png"):
    f.unlink()

# collect noisy frames
noisy_list = sorted(noisy_frames_dir.glob("*.png"))
assert len(noisy_list) > 0, "No noisy frames found!"

# copy to Restormer input dir
for img_path in tqdm(noisy_list, desc="Copying noisy frames"):
    shutil.copy(img_path, demo_noisy_dir / img_path.name)

print(f"Copied {len(noisy_list)} noisy frames to: {demo_noisy_dir}")


Copying noisy frames: 100%|██████████| 49/49 [00:00<00:00, 1122.07it/s]

Copied 49 noisy frames to: /content/Restormer/demo/noisy_frames


In [77]:
%cd /content/Restormer

!python demo.py \
  --task Gaussian_Color_Denoising \
  --input_dir ./demo/noisy_frames \
  --result_dir ./demo/restormer_frames

%cd /content


print("Restormer output frames:", len(list(rest_frames_dir.glob("*.png"))))

/content/Restormer

 ==> Running Gaussian_Color_Denoising with weights Denoising/pretrained_models/gaussian_color_denoising_blind.pth
 
100% 49/49 [01:00<00:00,  1.23s/it]

Restored images are saved at ./demo/restormer_frames/Gaussian_Color_Denoising
/content
Restormer output frames: 49


In [78]:
rest_list = sorted(rest_frames_dir.glob("*.png"))    # อ่านรายชื่อเฟรม Restormer ทั้งหมด และจัดเรียงตามลำดับไฟล์
print("Number of Restormer frames:", len(rest_list))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_rest = cv2.VideoWriter(str(rest_video_path), fourcc, fps, (width, height))

for img_path in rest_list:      # วนรวมทุกเฟรมของ Restormer กลับเป็นวิดีโอเดียว
    frame = cv2.imread(str(img_path))
    if frame is None:
        print("Warning: cannot read", img_path)
        continue
    frame = cv2.resize(frame, (width, height))    # ปรับขนาดเฟรมให้ตรงกับวิดีโอต้นฉบับ
    out_rest.write(frame)

out_rest.release()
print("Saved Restormer video to:", rest_video_path)

Number of Restormer frames: 49
Saved Restormer video to: /content/data/restormer.mp4


In [79]:
classical_dir = gaussian_frames_dir   # หรือ median_frames_dir

triplet_video_path = base_dir / "data" / "comparison_triplet.mp4"

clean_names = sorted([p.name for p in clean_frames_dir.glob("*.png")])

print("Clean:", len(clean_names))
print("Classical:", len(list(classical_dir.glob('*.png'))))
print("Restormer:", len(list(rest_frames_dir.glob('*.png'))))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_triplet = cv2.VideoWriter(str(triplet_video_path), fourcc, fps, (3 * width, height))

for img_name in tqdm(clean_names):
    clean     = cv2.imread(str(clean_frames_dir / img_name))
    classical = cv2.imread(str(classical_dir   / img_name))
    rest      = cv2.imread(str(rest_frames_dir / img_name))

    # ถ้าไฟล์ไหนหาย ข้าม
    if clean is None or classical is None or rest is None:
        print("Missing frame:", img_name)
        continue

    clean     = cv2.resize(clean, (width, height))
    classical = cv2.resize(classical, (width, height))
    rest      = cv2.resize(rest, (width, height))

    concat = np.hstack([clean, classical, rest])

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(concat, "Original",  (20, 40),          font, 1, (0, 255, 0), 2)
    cv2.putText(concat, "Classical", (width + 20, 40),  font, 1, (0, 255, 0), 2)
    cv2.putText(concat, "Restormer", (2*width + 20, 40),font, 1, (0, 255, 0), 2)

    out_triplet.write(concat)

out_triplet.release()
print("Saved comparison triplet video to:", triplet_video_path)

Clean: 49
Classical: 49
Restormer: 49


100%|██████████| 49/49 [00:02<00:00, 24.03it/s]

Saved comparison triplet video to: /content/data/comparison_triplet.mp4


In [80]:
# โฟลเดอร์สำหรับเก็บผลเชิงคุณภาพ
qual_dir = base_dir / "data" / "qualitative_results"
qual_dir.mkdir(parents=True, exist_ok=True)

# ใช้จำนวนเฟรมจาก clean_frames เป็นอ้างอิง
all_clean = sorted(clean_frames_dir.glob("*.png"))
total_frames = len(all_clean)
print("Total frames:", total_frames)

# เลือก 5 เฟรมแบบกระจายเท่า ๆ กัน
sample_indices = np.linspace(0, total_frames - 1, 5, dtype=int)
print("Sample frame indices:", sample_indices)

for idx in sample_indices:
    name = f"{idx:05d}.png"

    orig  = cv2.imread(str(clean_frames_dir    / name))
    noisy = cv2.imread(str(noisy_frames_dir    / name))
    gauss = cv2.imread(str(gaussian_frames_dir / name))
    med   = cv2.imread(str(median_frames_dir   / name))
    rest  = cv2.imread(str(rest_frames_dir     / name))

    # ถ้าเฟรมไหนหาย ข้าม
    if orig is None or noisy is None or gauss is None or med is None or rest is None:
        print("Missing frame:", name)
        continue
    panel = np.hstack([orig, noisy, gauss, med, rest])
    panel = add_labels(panel)
    cv2.imwrite(str(qual_dir / f"frame_{idx:05d}_5panel.png"), panel)
print("Saved qualitative PNGs to:", qual_dir)

Total frames: 49
Sample frame indices: [ 0 12 24 36 48]
Saved qualitative PNGs to: /content/data/qualitative_results


In [81]:
# Calculated PSNR and SSIM

def eval_folder(ref_dir: Path, test_dir: Path):
    psnrs, ssims = [], []
    names = sorted([p.name for p in ref_dir.glob("*.png")])

    for img_name in names:
        ref_path  = ref_dir  / img_name
        test_path = test_dir / img_name

        if not test_path.exists():
            continue

        ref  = cv2.imread(str(ref_path))
        test = cv2.imread(str(test_path))

        if ref is None or test is None:
            continue

        ref  = cv2.cvtColor(ref,  cv2.COLOR_BGR2RGB)
        test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)

        psnr = peak_signal_noise_ratio(ref, test, data_range=255)
        ssim = structural_similarity(ref, test, channel_axis=2, data_range=255)

        psnrs.append(psnr)
        ssims.append(ssim)

    return float(np.mean(psnrs)), float(np.mean(ssims))


clean_dir = clean_frames_dir

psnr_noisy,   ssim_noisy   = eval_folder(clean_dir, noisy_frames_dir)
psnr_med,     ssim_med     = eval_folder(clean_dir, median_frames_dir)
psnr_gauss,   ssim_gauss   = eval_folder(clean_dir, gaussian_frames_dir)
psnr_rest,    ssim_rest    = eval_folder(clean_dir, rest_frames_dir)

print("Quantitative Results (vs clean)")
print(f"Noisy      PSNR / SSIM : {psnr_noisy:.3f}  /  {ssim_noisy:.4f}")
print(f"Median     PSNR / SSIM : {psnr_med:.3f}    /  {ssim_med:.4f}")
print(f"Gaussian   PSNR / SSIM : {psnr_gauss:.3f}  /  {ssim_gauss:.4f}")
print(f"Restormer  PSNR / SSIM : {psnr_rest:.3f}   /  {ssim_rest:.4f}")

Quantitative Results (vs clean)
Noisy      PSNR / SSIM : 20.836  /  0.3414
Median     PSNR / SSIM : 27.718    /  0.7241
Gaussian   PSNR / SSIM : 28.380  /  0.7031
Restormer  PSNR / SSIM : 31.508   /  0.8318


In [82]:
import pandas as pd

records = []

# Define common frame names from clean_frames_dir
common = sorted([p.name for p in clean_frames_dir.glob("*.png")])

for name in tqdm(common, desc="Calculating per-frame metrics"):
    clean_img_path = clean_frames_dir / name
    noisy_img_path = noisy_frames_dir / name
    gaussian_img_path = gaussian_frames_dir / name
    median_img_path = median_frames_dir / name
    restormer_img_path = rest_frames_dir / name

    clean = cv2.imread(str(clean_img_path))
    noisy = cv2.imread(str(noisy_img_path))
    gaussian = cv2.imread(str(gaussian_img_path))
    median = cv2.imread(str(median_img_path))
    restormer = cv2.imread(str(restormer_img_path))

    # Ensure all images are loaded before proceeding
    if all(img is not None for img in [clean, noisy, gaussian, median, restormer]):
        # Convert to RGB for SSIM calculation, as structural_similarity expects color channels
        clean_rgb = cv2.cvtColor(clean, cv2.COLOR_BGR2RGB)
        noisy_rgb = cv2.cvtColor(noisy, cv2.COLOR_BGR2RGB)
        gaussian_rgb = cv2.cvtColor(gaussian, cv2.COLOR_BGR2RGB)
        median_rgb = cv2.cvtColor(median, cv2.COLOR_BGR2RGB)
        restormer_rgb = cv2.cvtColor(restormer, cv2.COLOR_BGR2RGB)

        records.append({
            "frame": name,
            "psnr_noisy": peak_signal_noise_ratio(clean_rgb, noisy_rgb, data_range=255),
            "psnr_gaussian": peak_signal_noise_ratio(clean_rgb, gaussian_rgb, data_range=255),
            "psnr_median": peak_signal_noise_ratio(clean_rgb, median_rgb, data_range=255),
            "psnr_restormer": peak_signal_noise_ratio(clean_rgb, restormer_rgb, data_range=255),
            "ssim_noisy": structural_similarity(clean_rgb, noisy_rgb, channel_axis=2, data_range=255),
            "ssim_gaussian": structural_similarity(clean_rgb, gaussian_rgb, channel_axis=2, data_range=255),
            "ssim_median": structural_similarity(clean_rgb, median_rgb, channel_axis=2, data_range=255),
            "ssim_restormer": structural_similarity(clean_rgb, restormer_rgb, channel_axis=2, data_range=255),
        })
    else:
        print(f"Warning: Missing one or more images for frame {name}. Skipping.")

df = pd.DataFrame(records)
print("Per-frame metrics DataFrame created.")


Calculating per-frame metrics: 100%|██████████| 49/49 [00:26<00:00,  1.87it/s]

Per-frame metrics DataFrame created.


In [83]:
df.to_csv("score_result.csv" , index = False)